<a href="https://colab.research.google.com/github/noerbarry/validasi_nomor_wa/blob/main/Validasi_nomor_WA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv
import os

file_path = "/content/uploadnomor/nomor.csv"  # Tentukan jalur file

# Periksa apakah file ada
if not os.path.exists(file_path):
    print("Error: file nomor.csv tidak ditemukan!")
    exit(1)

# Baca file CSV baris per baris
with open(file_path, "r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Ekstrak nomor telepon dari kolom 'Phone_Number'
        number = row['Phone_Number'].strip()  # Hapus spasi di awal/akhir jika ada

        # Validasi nomor dengan melakukan permintaan ke API WhatsApp (Simulasi permintaan)
        # Menyimulasikan respons yang berbeda untuk tujuan demonstrasi
        respons_simulasi = requests.get(f"https://api.whatsapp.com/send?phone={number}&text=test")

        if respons_simulasi.status_code == 200:
            # Simulasi teks respons API untuk beberapa kasus
            if int(number) % 2 == 0:
                print(f"{number} aktif di WhatsApp")
            else:
                print(f"{number} tidak aktif di WhatsApp")
        else:
            print(f"Terjadi kesalahan saat memeriksa {number}: Gagal terhubung ke API WhatsApp")

In [ ]:
import csv
import requests

# Fungsi untuk memvalidasi nomor WhatsApp
def validate_whatsapp_number(number):
    url = "https://pabar-validasi-nomor-wa.vercel.app/api/validate"
    params = {
        "target": number,
        "countryCode": "62"
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return f"Request failed for number {number} with status code: {response.status_code}"

# Baca nomor telepon dari file CSV
csv_file_path = '/content/nomor.csv'  # Ganti dengan path file CSV Anda
with open(csv_file_path, 'r') as file:
    reader = csv.reader(file)
    header_skipped = False
    for row in reader:
        if not header_skipped:
            header_skipped = True
            continue  # Lewati header jika ada

        if len(row) > 0:  # Pastikan baris memiliki setidaknya satu elemen
            phone_number = row[0]  # Ambil nomor telepon dari kolom pertama (atau sesuaikan dengan kolom yang sesuai)
            result = validate_whatsapp_number(phone_number)
            print(f"Nomor: {phone_number} - Response: {result}")

In [ ]:
import csv
import requests

# Fungsi untuk memvalidasi nomor WhatsApp
def validate_whatsapp_number(number):
    url = "https://pabar-validasi-nomor-wa.vercel.app/api/validate"
    params = {
        "target": number,
        "countryCode": "62"
    }

    try:
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            if data.get('status') == True:
                not_registered = data.get('not_registered')
                registered = data.get('registered')

                if not_registered is not None and registered is not None:
                    if len(not_registered) == 0 and len(registered) > 0:
                        return "registered"
                    elif len(registered) == 0 and len(not_registered) == 0:
                        return "unknown"
                    else:
                        return "not_registered"
                else:
                    return "invalid format number"
            else:
                return f"Response: {data}"
        else:
            return f"Request failed with status code {response.status_code}"
    except requests.RequestException as e:
        return f"Connection Error: {str(e)}"

# Baca nomor telepon dari file CSV dan tulis ke file CSV baru
csv_file_path = '/content/nomor.csv'  # Ganti dengan path file CSV Anda
output_csv_file_path = '/content/hasil_validasi_nomor.csv'  # Ganti dengan path untuk menyimpan hasil
max_numbers_to_check = 400  # Jumlah maksimal nomor yang akan diperiksa

with open(csv_file_path, 'r') as file, open(output_csv_file_path, 'w', newline='') as output_file:
    reader = csv.reader(file)
    writer = csv.writer(output_file)

    writer.writerow(['nomor', 'status'])  # Menulis header

    header_skipped = False
    count_numbers_checked = 0  # Hitung jumlah nomor yang telah diperiksa
    for row in reader:
        if not header_skipped:
            header_skipped = True
            continue  # Lewati header jika ada

        if count_numbers_checked >= max_numbers_to_check:
            print(f"Jumlah nomor yang diperiksa sudah mencapai batas maksimal ({max_numbers_to_check}).")
            break

        if len(row) > 0:  # Pastikan baris memiliki setidaknya satu elemen
            phone_number = row[0]  # Ambil nomor telepon dari kolom pertama (atau sesuaikan dengan kolom yang sesuai)
            result = validate_whatsapp_number(phone_number)

            # Tulis hasil ke file CSV baru
            writer.writerow([phone_number, result])
            print(f"Nomor: {phone_number} - Status: {result}")

            count_numbers_checked += 1  # Tambah jumlah nomor yang telah diperiksa

print("Proses selesai. Hasil validasi disimpan di", output_csv_file_path)
